In [1]:
import finetune_resnet
import h5py
from keras.metrics import sparse_categorical_accuracy
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

C:\tools\miniconda3\lib\site-packages\h5py\__init__.py:75: UserWarning: h5py is running against HDF5 1.10.5 when it was built against 1.10.4, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
dataset = h5py.File("../Market-1501/market.h5")
x = dataset["train_images"]
attributes_target = dataset["train_labels"]
file_names = dataset["train_images_files"]
ids_encoder = dict()
train_ids = np.zeros(len(file_names), dtype=int)
i = 0
for j, file in enumerate(file_names):
    if file[:4] not in ids_encoder:
        ids_encoder[file[:4]] = i
        i += 1
    train_ids[j] = ids_encoder[file[:4]]
n_id = len(ids_encoder)

In [3]:
train_ids.shape, n_id

((12567,), 751)

In [4]:
_, training_model, inference_model = finetune_resnet.get_models(n_person=n_id)














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
training_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 64, 3)   0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 2048)         23587712    input_1[0][0]                    
__________________________________________________________________________________________________
attributes (Dense)              (None, 27)           55323       resnet50[1][0]                   
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 512)          1049088     resnet50[1][0]                   
__________________________________________________________________________________________________
dense_1 (D

In [6]:
# sparse_top_k_categorical_accuracy
# TODO: rename layers
training_model.compile("adam", loss=["binary_crossentropy", "sparse_categorical_crossentropy"], metrics={"attributes":[finetune_resnet.market_attribute_accuracy], "ids":[sparse_categorical_accuracy]})



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
attributes, categorical_id = training_model.predict(finetune_resnet.preprocess_images(x[:1]))
attributes.shape, categorical_id.shape

((1, 27), (1, 751))

In [9]:
id_embedding = inference_model.predict(finetune_resnet.preprocess_images(x[:1]))
id_embedding.shape

(1, 539)

In [10]:
# training_model.load_weights("../models/")
cb = [
    ModelCheckpoint("../models/full-model-{val_loss:.2f}.h5"),
    EarlyStopping(patience=5)
]

# n_samples = 400
epochs = 60
batch_size = 32

history = training_model.fit(
    x=finetune_resnet.preprocess_images(np.array(x)),
    y=[finetune_resnet.preprocess_labels(np.array(attributes_target)), train_ids],
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=cb,
)



Train on 10053 samples, validate on 2514 samples
Epoch 1/60
 1696/10053 [====>.........................] - ETA: 28:59 - loss: 7.4236 - attributes_loss: 0.4715 - ids_loss: 6.9521 - attributes_market_attribute_accuracy: 0.6923 - ids_sparse_categorical_accuracy: 0.0124

KeyboardInterrupt: 

In [10]:
history.history

{'val_loss': [8.960886764526368,
  8.34894905090332,
  8.428741073608398,
  8.925591278076173,
  9.473195648193359],
 'val_attributes_loss': [0.6108266234397888,
  0.4874896168708801,
  0.4583412766456604,
  0.4167355537414551,
  0.44159516096115115],
 'val_ids_loss': [8.350060081481933,
  7.861459159851075,
  7.970399856567383,
  8.508855628967286,
  9.03160057067871],
 'val_attributes_market_attribute_accuracy': [0.6235227584838867,
  0.6647727727890015,
  0.6753409624099731,
  0.6856818199157715,
  0.6978409767150879],
 'val_ids_sparse_categorical_accuracy': [0.0, 0.0, 0.0, 0.0125, 0.0125],
 'loss': [5.15276882648468,
  1.176661390066147,
  0.3717037051916122,
  0.1464253045618534,
  0.056865626201033595],
 'attributes_loss': [0.4249697357416153,
  0.1777344048023224,
  0.09251592755317688,
  0.0410867303609848,
  0.02362410519272089],
 'ids_loss': [4.727799129486084,
  0.9989269793033599,
  0.2791877768933773,
  0.10533857569098473,
  0.03324152128770948],
 'attributes_market_attri

In [11]:
# test function with query - gallery
test_data = np.array([
    x[0], x[1], x[2], x[100], x[200]
])
res = inference_model.predict(finetune_resnet.preprocess_images(test_data))

In [12]:
# 0 1 2 are the same person, 3, 4 are different persons
dist = [
    np.linalg.norm(res[0] - res[1]),
    np.linalg.norm(res[0] - res[2]),
    np.linalg.norm(res[0] - res[3]),
    np.linalg.norm(res[0] - res[4]),
]
dist

[8.455841, 11.19874, 47.843533, 43.932945]

In [14]:
# test on unseen data
test_data = np.array([
    x[400], x[401], x[402], x[500], x[600]
])
res = inference_model.predict(finetune_resnet.preprocess_images(test_data))
# 0 1 2 are the same person, 3, 4 are different persons
dist = [
    np.linalg.norm(res[0] - res[1]),
    np.linalg.norm(res[0] - res[2]),
    np.linalg.norm(res[0] - res[3]),
    np.linalg.norm(res[0] - res[4]),
]
dist, (file_names[400], file_names[401], file_names[402], file_names[500], file_names[600])

([13.363515, 17.286472, 15.522013, 16.504707],
 ('0067_c5s1_009476_00.jpg',
  '0067_c5s1_019276_00.jpg',
  '0067_c5s1_019301_00.jpg',
  '0077_c4s1_018376_00.jpg',
  '0095_c4s1_015376_00.jpg'))

In [17]:
training_model.save_weights("../models/full-model-weights.h5")